In [49]:
config_names = [
    "4chan_meta_sep",
    "c4_100_domains",
    "c4_en",
    "dolma_100_programing_languages",
    "dolma_100_subreddits",
    "dolma-v1_5",
    "falcon-refinedweb",
    "gab",
    "m2d2_s2orc_unsplit",
    "m2d2_wikipedia_unsplit",
    "manosphere_meta_sep",
    "mc4",
    "ptb",
    "redpajama",
    "twitterAAE_HELM_fixed",
    "wikitext_103"
]

In [38]:
from datasets import load_dataset, Dataset
import os

In [51]:
examples = []
for config_name in config_names:
    sub_dataset = load_dataset("allenai/paloma", config_name, split='val', token=os.environ["HF_TOKEN"])

    shuffled_dataset = sub_dataset.shuffle(seed=42)  # Set seed for reproducibility
    examples.extend(shuffled_dataset.select(range(min(10, len(shuffled_dataset)))))

Resolving data files:   0%|          | 0/99 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/99 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/167 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/167 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

In [52]:
# shuffle examples
import random
random.shuffle(examples)

In [53]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-7b-0724-hf")

In [57]:
# Getting all of the tokens together 
chunk_size = 2048
from tqdm import tqdm

tokens = []
for example in tqdm(examples):
    _tokens = tokenizer.encode(example['text'])
    _tokens.append(tokenizer.eos_token_id)
    tokens.extend(_tokens)

chunks = [tokens[i:i+chunk_size] for i in range(0, len(tokens), chunk_size)]

if len(chunks[-1]) < chunk_size:
    chunks = chunks[:-1]

100%|██████████| 151/151 [00:10<00:00, 14.34it/s]


In [58]:
# shuffle chunks
random.shuffle(chunks)

In [66]:
# create a dataset from the first 1024 chunks 
dataset_data = chunks[:1024]

dataset_dict = [
    {"input_ids": chunk,
     "text": tokenizer.decode(chunk)
    } for chunk in dataset_data
]

dataset = Dataset.from_list(dataset_dict)

In [67]:
# save to huggingface as a dataset
dataset.push_to_hub("pico-lm/pretokenized-eval-batch",  token=os.environ["HF_TOKEN"], split="val")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/pico-lm/pretokenized-eval-batch/commit/e57e6b7d6694b9f8fd1a02a9b1339d03ff3fed76', commit_message='Upload dataset', commit_description='', oid='e57e6b7d6694b9f8fd1a02a9b1339d03ff3fed76', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/pico-lm/pretokenized-eval-batch', endpoint='https://huggingface.co', repo_type='dataset', repo_id='pico-lm/pretokenized-eval-batch'), pr_revision=None, pr_num=None)